In [1]:
import torch
import torchvision
from torchvision.datasets import mnist, MNIST
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F


In [2]:
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:

#datasets stage

train_data = MNIST(root = '/MNIST', download= True, train = True, transform = transform['train'])
test_data = MNIST(root = '/MNIST', download = False, train = False, transform = transform['val'])

In [4]:
#load datasets to dataloader

train_dl = DataLoader(train_data, batch_size = 256, shuffle = True)
test_dl = DataLoader(test_data, 256,shuffle =  True)

In [5]:
train_size = len(train_dl)
test_size = len(test_dl)

In [6]:
class Flatten(nn.Module):
    def  __init__(self):
        super(Flatten ,self).__init__()
    def forward(self,x):
        x = x.view(-1, x.size()[1] * x.size()[2] * x.size(3))
        return x

In [19]:
class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

In [20]:
net = Mnist_CNN().cuda()

In [21]:
print(net)

Mnist_CNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(16, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
)


In [22]:
def train(dataloader,model,criterion,optimizer,epochs):
    Loss = 0
    Acc = 0
    i = 0
    model.train()
    for epoch in range(epochs):
        Loss = 0
        Acc = 0
        for inputs, labels in dataloader:
            inputs = inputs.cuda()
            labels = labels.cuda()
           
            outputs = model(inputs)
            value, pred = torch.max(outputs.data,1)
            optimizer.zero_grad()
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            Loss += loss.item()
            Acc += (pred == labels.data).average().item()
        print(f'epoch : {epoch}, loss : {Loss/train_size}, acc : {Acc/train_size}')
       # torch.save(model.state_dict(),'./hymenoptera_data/hymenoptera_data/model.pth')
    return model

In [13]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(net.parameters())

In [23]:
train(train_dl, net, criterion, optimizer, 10)

ValueError: Expected input batch_size (16384) to match target batch_size (256).

In [ ]:
images, labels = next(iter(train_dl))